# Regresion Logistica: Deteccion de SPAM             

### En este ejercicio se muestran los fundamentos de la regresion logistica planteando uno de los primeros problemas que fueron solucionados mediante el uso de tecnicas de Machine Learning: Deteccion de SPAM.

## Enunciado del ejercicio.

### Se propone la construccion de un sistema automatico capaz de pedecir si un correo determinado es un correo SPAM o no, para ello se utilizara el siguiente DataSet. 

[2007 TREC Public SPam Corpus](terabox)

La carpeta contiene 75,419 mensajes de correo electronico.
25,220 son Ham 
50,199 son SPAM

These Messages Constitute al the messages delivered to a particular server betwwen these dates:

Sun, 8 Appr 2007 13:07:21 - 0400
Fri, 6 Jul 2007 07:04:53 - 0400


* Aprendizaje **Supervisado**.
* Aprendizaje **Basado en modelos**
* Se correponde con un **modelo lineal generalizado**
* Realiza predicciones computando una **suma ponderada de las caracteristicas de entrada** sumandole una constante conocida como bias, pero se aplica una funcion logistica al resultado.

### 1.-Funciones complementarias.

En este caso practico relacionado con la deteccion de correos electronicos de SPAM, el conjunto de datos que se dispone, esta formado por correos electronicos , con sus correspondientes cabeceras y campos adicionales. Por lo tanto requieren un prepocesamiento previo a que sean ingeridos por el algoritmo de Machine Learning.

In [1]:
# Esta clase Facilita el preprocesamiento de correos electronicos que poseen codigo HTML.
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict= False
        self.convert_charrefs = True
        self.fed = []
        
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)
    

In [2]:
#Funcion que se encargara de eliminar los tags HTML que se encuentren en el texto del correo.
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
#Ejemplo de eliminacion de los tags de HTML de un texto.
t = '<tr><td align="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los tags de HTML que se encuentren en el correo  electronico, deben realizarse otras acciones de preprocesamiento para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminaion de los signos de puntuacion, eliminacion de posibles campos de correo elecronico que no son relevantes o eliminacion de los afijos de una palabra manteniendo unicamente la raiz de la misma (Stemming). La clase que se muestra a continuacion realiza estas transformaciones. 

In [4]:
import email
import string
import nltk

class Parser:
    def _init_(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('English'))
        self.punctuation = list(string.punctuation)
        
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
        
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg ['Subject'] else[]
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return{"Subject": subject,
               "body": body,
               "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extact the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload is list):
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body
                                            
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions,
        clean the punctuation symbols and stemming of the text."""
        for c in self.punctuation:
            text = text.replace(c, " ")
        text = text.replace("\t"," ")
        text = text.replace("\n"," ")
        tokens = list(filter(None, text.split(" ")))
        #Stemming of the tokens
        return[self-stemmer.stem(w) for w in tokens if w not in self.stopwords]

# Lectura de un correo en formato raw

In [5]:
inmail = open("datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

#### Lectura del indice.

In [10]:
import nltk
p = Parser()
p.parse("datasets/trec07p/data/inmail.1")

AttributeError: 'Parser' object has no attribute 'punctuation'

# Lectura del indice

Estas funciones complementarias se encargan de cargar en memoria la ruta de cada correo electronico y su etiqueta correspondiente {spam,ham}

In [ ]:
index = open("datasets/trec07p/full/index").readlines()
index

In [ ]:
import os 

DATASET_PATH = "datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range (n_elements):
        mail = index[i].split("../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.appen({"label":label, "email_path": os.path.join(DATASET_PATH,path)})
        return ret_indexes
   

In [ ]:
def parse_email(index):
    p= Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]
    

In [ ]:
indexes = parse_index("datasets/trec07p/full/index", 10)
indexes

# 2.-Prepocesamiento de datos

Con las funciones presentadas anteriormente se permite la lectura de los correos electronicos de manera programatica y el procesamiento de los mismos para eliminar aquellos componentes que no resulten en utilidad para la deteccion de correos SPAM , sin embargo cada uno de los correos sigue estando representado por un diccionario de Python con una serie de palabras .

In [ ]:
#Cargar el indice y las etiquetas en memoria.
index = parse_index("datasets/trec07p/full/index",1)

In [ ]:
#Leer el primer correo
import os

open(index[0]["email_path"]).read()

In [ ]:
#Parsear al primer correo
mail, label = parse_email(index[0])
